# 990PF 

In [7]:
import pandas as pd
from xml.etree import cElementTree as ElementTree
import xml.etree.ElementTree as et
from tqdm import tqdm
import logging
import re

logging.basicConfig(format='%(asctime)s: %(message)s', filename='analysis.log', level=logging.DEBUG)

In [8]:
class XmlListConfig(list):
    def __init__(self, aList):
        for element in aList:
            if element:
                # treat like dict
                if len(element) == 1 or element[0].tag != element[1].tag:
                    self.append(XmlDictConfig(element))
                # treat like list
                elif element[0].tag == element[1].tag:
                    self.append(XmlListConfig(element))
            elif element.text:
                text = element.text.strip()
                if text:
                    self.append(text)


class XmlDictConfig(dict):
    '''
    Example usage:

    >>> tree = ElementTree.parse('your_file.xml')
    >>> root = tree.getroot()
    >>> xmldict = XmlDictConfig(root)

    Or, if you want to use an XML string:

    >>> root = ElementTree.XML(xml_string)
    >>> xmldict = XmlDictConfig(root)

    And then use xmldict for what it is... a dict.
    '''
    def __init__(self, parent_element):
        if parent_element.items():
            self.update(dict(parent_element.items()))
        for element in parent_element:
            if element:
                # treat like dict - we assume that if the first two tags
                # in a series are different, then they are all different.
                if len(element) == 1 or element[0].tag != element[1].tag:
                    aDict = XmlDictConfig(element)
                # treat like list - we assume that if the first two tags
                # in a series are the same, then the rest are the same.
                else:
                    # here, we put the list in dictionary; the key is the
                    # tag name the list elements all share in common, and
                    # the value is the list itself 
                    aDict = {element[0].tag: XmlListConfig(element)}
                # if the tag has attributes, add those to the dict
                if element.items():
                    aDict.update(dict(element.items()))
                self.update({element.tag: aDict})
            # this assumes that if you've got an attribute in a tag,
            # you won't be having any text. This may or may not be a 
            # good idea -- time will tell. It works for the way we are
            # currently doing XML configuration files...
            elif element.items():
                self.update({element.tag: dict(element.items())})
            # finally, if there are no child tags and no attributes, extract
            # the text
            else:
                self.update({element.tag: element.text})

def get_field(xmldict, path=None, default=None):
    if path is None: return xmldict
    try:
        for p in path:
            # Ugly namespaces are ugly, let's deal with them here
            xmldict = xmldict['{http://www.irs.gov/efile}' + p]
        return xmldict
    except KeyError:
        # If this key does not exist in the dictionary, 
        return default
        # Note: it may be faster to check ```if p not in dict``` rather
        # than set up this try/except block. I think it depends on how
        # often this fails - the more it fails, the worse the try/except does.

In [7]:
tree = et.parse("data/test/201722789349300202_public.xml")
root = tree.getroot()
#Converting from xml to dictionary
xmldict = XmlDictConfig(root)


In [17]:
def analyze_one(path):
    object_id = re.sub("\D", "", path)
    object_list = [[object_id]]
    object_df = pd.DataFrame(object_list, columns =['Object_ID'])
    tree = et.parse(path)
    root = tree.getroot()
    #Converting from xml to dictionary
    xmldict = XmlDictConfig(root)

    
    ##############Header Information 
    #Date Information
    TaxPeriodBeginDt = get_field(xmldict, ['ReturnHeader','TaxPeriodBeginDt'])
    TaxPeriodEndDt = get_field(xmldict, ['ReturnHeader','TaxPeriodEndDt'])
    
    
    #Filer Information
    filer_data = get_field(xmldict, ['ReturnHeader', 'Filer'], {})
    EIN = get_field(filer_data, ['EIN'])
    BusinessName = get_field(filer_data, ['BusinessName', 'BusinessNameLine1Txt'])
    City = get_field(filer_data, ['USAddress', 'CityNm'])
    State = get_field(filer_data, ['USAddress', 'StateAbbreviationCd'])
    ZIPCd = get_field(filer_data, ['USAddress', 'ZIPCd'])
    TaxYr = get_field(xmldict, ['ReturnHeader', 'TaxYr'])
    filer_list = [[EIN, BusinessName,TaxPeriodBeginDt, TaxPeriodEndDt,TaxYr, City, State, ZIPCd]]
    filer_df = pd.DataFrame(filer_list, columns = ['EIN', 'BusinessName','TaxPeriodBeginDt','TaxPeriodEndDt','TaxYr','City', 'State', 'ZIPCd']) 
    
    
    ##############Return Data
    return_data = get_field(xmldict, ['ReturnData', 'IRS990PF'])
    if return_data is None:
        logging.warning(f'Could not find Return Data for file: {path}')
        return None
    
    #AddressChangeInd
    AddressChangeInd = get_field(return_data, ['AddressChangeInd'], "")
    
    #InitialReturnInd
    InitialReturnInd = get_field(return_data, ['InitialReturnInd'], "")
    
    #InitialReturnFormerPubChrtyInd
    InitialReturnFormerPubChrtyInd = get_field(return_data, ['InitialReturnFormerPubChrtyInd'], "")
    
    #AmendedReturnInd
    AmendedReturnInd = get_field(return_data, ['AmendedReturnInd'], "")
    
    #FinalReturnInd
    FinalReturnInd = get_field(return_data, ['FinalReturnInd'], "")
    
    #NameChange this may only be for paper filers 
    NameChange = get_field(return_data, ['NameChange'], "")
    
    #PFStatusTermSect507b1AInd
    PFStatusTermSect507b1AInd = get_field(return_data, ['PFStatusTermSect507b1AInd'], "")
    
    #Organization501c3ExemptPFInd
    Organization501c3ExemptPFInd = get_field(return_data, ['Organization501c3ExemptPFInd'], "")
    
    #Organization4947a1TrtdPFInd
    Organization4947a1TrtdPFInd = get_field(return_data, ['Organization4947a1TrtdPFInd'], "")
    
    #Organization501c3TaxablePFInd
    Organization501c3TaxablePFInd = get_field(return_data, ['Organization501c3TaxablePFInd'], "")
    
     #End of year assets "FMV of Assets at End of Year (required)"
    FMVAssetsEOYAmt = get_field(return_data, ['FMVAssetsEOYAmt'], "NA")
    
    #AdjustedNetIncomeAmt
    AdjustedNetIncomeAmt = get_field(return_data, ['AnalysisOfRevenueAndExpenses','AdjustedNetIncomeAmt'], "")

    #TotalRevAndExpnssAmt
    TotalRevAndExpnssAmt = get_field(return_data, ['AnalysisOfRevenueAndExpenses','TotalRevAndExpnssAmt'], "")
    
    #TotalNetInvstIncmAmt
    TotalNetInvstIncmAmt = get_field(return_data, ['AnalysisOfRevenueAndExpenses','TotalNetInvstIncmAmt'], "")

    #TotalAdjNetIncmAmt
    TotalAdjNetIncmAmt = get_field(return_data, ['AnalysisOfRevenueAndExpenses','TotalAdjNetIncmAmt'], "")
    
    #TotalExpensesRevAndExpnssAmt 'Form990PFBalanceSheetsGrp',
    TotalExpensesRevAndExpnssAmt = get_field(return_data, ['AnalysisOfRevenueAndExpenses','TotalExpensesRevAndExpnssAmt'], "")
    
    #TotalExpensesNetInvstIncmAmt
    TotalExpensesNetInvstIncmAmt = get_field(return_data, ['AnalysisOfRevenueAndExpenses','TotalExpensesNetInvstIncmAmt'], "")
    
    #TotalExpensesAdjNetIncmAmt
    TotalExpensesAdjNetIncmAmt = get_field(return_data, ['AnalysisOfRevenueAndExpenses','TotalExpensesAdjNetIncmAmt'], "")
    
    #TotalExpensesDsbrsChrtblAmt
    TotalExpensesDsbrsChrtblAmt = get_field(return_data, ['AnalysisOfRevenueAndExpenses','TotalExpensesDsbrsChrtblAmt'], "")
    
    #ExcessRevenueOverExpensesAmt
    ExcessRevenueOverExpensesAmt = get_field(return_data, ['AnalysisOfRevenueAndExpenses','ExcessRevenueOverExpensesAmt'], "")
    
    #NetInvestmentIncomeAmt
    NetInvestmentIncomeAmt = get_field(return_data, ['AnalysisOfRevenueAndExpenses','NetInvestmentIncomeAmt'], "")
    
    #AdjustedNetIncomeAmt
    AdjustedNetIncomeAmt = get_field(return_data, ['AnalysisOfRevenueAndExpenses','AdjustedNetIncomeAmt'], "")
    
    #TotalAssetsBOYAmt
    TotalAssetsBOYAmt = get_field(return_data, ['AnalysisOfRevenueAndExpenses','AdjustedNetIncomeAmt'], "")
    
    #TotalAssetsEOYAmt
    TotalAssetsEOYAmt = get_field(return_data, ['Form990PFBalanceSheetsGrp','TotalAssetsEOYAmt'], "")
    
    #TotalAssetsEOYFMVAmt
    TotalAssetsEOYFMVAmt = get_field(return_data, ['Form990PFBalanceSheetsGrp','TotalAssetsEOYFMVAmt'], "")
    
    #TotalLiabilitiesBOYAmt
    TotalLiabilitiesBOYAmt = get_field(return_data, ['Form990PFBalanceSheetsGrp','TotalLiabilitiesBOYAmt'], "")
    
    #TotalLiabilitiesEOYAmt
    TotalLiabilitiesEOYAmt = get_field(return_data, ['Form990PFBalanceSheetsGrp','TotalLiabilitiesEOYAmtt'], "")
    
    #TotNetAstOrFundBalancesBOYAmt
    TotNetAstOrFundBalancesBOYAmt = get_field(return_data, ['Form990PFBalanceSheetsGrp','TotNetAstOrFundBalancesBOYAmt'], "")
    
    #TotNetAstOrFundBalancesEOYAmt
    TotNetAstOrFundBalancesEOYAmt = get_field(return_data, ['Form990PFBalanceSheetsGrp','TotNetAstOrFundBalancesEOYAmt'], "")
    
    #AdjustedQualifyingDistriAmt
    AdjustedQualifyingDistriAmt = get_field(return_data, ['QualifyingDistriPartXIIGrp','AdjustedQualifyingDistriAmt'], "")
    
    #ExcessDistriCyovToNextYrAmt
    try:
        ExcessDistriCyovToNextYrAmt = get_field(return_data, ['UndistributedIncomeGrp','ExcessDistriCyovToNextYrAmt'], "")
        
    except TypeError:
        ExcessDistriCyovToNextYrAmt = "" 
    

    return_list = [[    AddressChangeInd,
                        InitialReturnInd,
                        InitialReturnFormerPubChrtyInd,
                        AmendedReturnInd,
                        FinalReturnInd,
                        NameChange,
                        PFStatusTermSect507b1AInd,
                        Organization501c3ExemptPFInd,
                        Organization4947a1TrtdPFInd,
                        Organization501c3TaxablePFInd,
                        FMVAssetsEOYAmt,
                        AdjustedNetIncomeAmt,
                        TotalRevAndExpnssAmt,
                        TotalNetInvstIncmAmt,
                        TotalAdjNetIncmAmt,
                        TotalExpensesRevAndExpnssAmt,
                        TotalExpensesNetInvstIncmAmt,
                        TotalExpensesAdjNetIncmAmt,
                        TotalExpensesDsbrsChrtblAmt,
                        ExcessRevenueOverExpensesAmt,
                        NetInvestmentIncomeAmt,
                        AdjustedNetIncomeAmt,
                        TotalAssetsBOYAmt,
                        TotalAssetsEOYAmt,
                        TotalAssetsEOYFMVAmt,
                        TotalLiabilitiesBOYAmt,
                        TotalLiabilitiesEOYAmt,
                        TotNetAstOrFundBalancesBOYAmt,
                        TotNetAstOrFundBalancesEOYAmt,
                        AdjustedQualifyingDistriAmt,
                        ExcessDistriCyovToNextYrAmt
                   ]]
    
    return_df = pd.DataFrame(return_list, columns =
                             ['AddressChangeInd',
                            'InitialReturnInd',
                            'InitialReturnFormerPubChrtyInd',
                            'AmendedReturnInd',
                            'FinalReturnInd',
                            'NameChange',
                            'PFStatusTermSect507b1AInd',
                            'Organization501c3ExemptPFInd',
                            'Organization4947a1TrtdPFInd',
                            'Organization501c3TaxablePFInd',
                            'FMVAssetsEOYAmt',
                            'AdjustedNetIncomeAmt',
                            'TotalRevAndExpnssAmt',
                            'TotalNetInvstIncmAmt',
                            'TotalAdjNetIncmAmt',
                            'TotalExpensesRevAndExpnssAmt',
                            'TotalExpensesNetInvstIncmAmt',
                            'TotalExpensesAdjNetIncmAmt',
                            'TotalExpensesDsbrsChrtblAmt',
                            'ExcessRevenueOverExpensesAmt',
                            'NetInvestmentIncomeAmt',
                            'AdjustedNetIncomeAmt',
                            'TotalAssetsBOYAmt',
                            'TotalAssetsEOYAmt',
                            'TotalAssetsEOYFMVAmt',
                            'TotalLiabilitiesBOYAmt',
                            'TotalLiabilitiesEOYAmt',
                            'TotNetAstOrFundBalancesBOYAmt',
                            'TotNetAstOrFundBalancesEOYAmt',
                            'AdjustedQualifyingDistriAmt',
                            'ExcessDistriCyovToNextYrAmt'])

    #Concatenating header information and contributions
    return pd.concat([object_df, filer_df, return_df], axis=1, ignore_index=False)
    

In [12]:
#This is the method I used to get all the files in the directory

import os

'''
    For the given path, get the List of all files in the directory tree 
'''
def getListOfFiles(dirName):
    # create a list of file and sub directories 
    # names in the given directory 
    listOfFile = os.listdir(dirName)
    allFiles = list()
    # Iterate over all the entries
    for entry in listOfFile:
        # Create full path
        fullPath = os.path.join(dirName, entry)
        # If entry is a directory then get the list of files in this directory 
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
                
    return allFiles


dirName = 'C:/Users/Administrator/Desktop/irs_2018';
files = getListOfFiles(dirName)
files = files[0:235160]

In [13]:
df = pd.concat([one_row for name in tqdm(files) if (one_row := analyze_one(name)) is not None])
df


100%|██████████| 235160/235160 [12:31<00:00, 313.11it/s]


,Object_ID,EIN,BusinessName,TaxPeriodBeginDt,TaxPeriodEndDt,TaxYr,City,State,ZIPCd,AddressChangeInd,...,AdjustedNetIncomeAmt,TotalAssetsBOYAmt,TotalAssetsEOYAmt,TotalAssetsEOYFMVAmt,TotalLiabilitiesBOYAmt,TotalLiabilitiesEOYAmt,TotNetAstOrFundBalancesBOYAmt,TotNetAstOrFundBalancesEOYAmt,AdjustedQualifyingDistriAmt,ExcessDistriCyovToNextYrAmt
0,2018201800089349100000,275023155,Global Benefits Group (GBG) Foundation,2016-01-01,2016-12-31,2016,Wilmington,DE,198091377,,...,,,523692,523692,0,,29115,523692,17953,78421
0,2018201800089349100005,391764134,BIOPHARMACEUTICAL TECHNOLOGY CENTER,2016-09-01,2017-08-31,2016,MADISON,WI,53711,,...,0,0,198716,290689,13056,,182190,192820,488647,
0,2018201800089349100010,357014492,THE CRAVENS FOUNDATION,2016-07-01,2017-06-30,2016,BROOKLYN,NY,11215,,...,,,1892135,0,250,,1415048,1891885,233567,50486
0,2018201800089349100050,205854888,Hecker Family Foundation,2016-11-01,2017-10-31,2016,St Louis,MO,63105,,...,,,14372,14372,0,,14434,14372,0,5184397
0,2018201800089349100055,113170326,CENTER FOR ESSENTIAL MANAGEMENT SERVICES INC,2016-11-01,2017-10-31,2016,CENTERPORT,NY,11721,,...,0,0,16982,16982,5594,,9249,13282,121801,983998
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,2018201822009349100602,204094041,EVANOW PUBLISHING INC,2017-01-01,2017-12-31,2017,CONROE,TX,77303,,...,,,1967339,1967339,0,,1527440,1967339,134649,1375132
0,2018201822009349100607,261328308,THE COACH MARTY JOHNSON FOUNDATION,2017-01-01,2017-12-31,2017,HUNTINGTON BEACH,CA,92646,,...,7,7,26974,43057,,,31977,26974,5975,31875
0,2018201822009349100612,870713088,CHRISTIAN IMPACT INC,2017-01-01,2017-12-31,2017,LONG BEACH,CA,90803,X,...,,,1033327,1033327,0,,874450,1033327,469523,1265810
0,2018201822009349100702,133973868,WAKA FOUNDATION INC,2016-09-01,2017-08-31,2016,NEW YORK,NY,10036,,...,,,228090,327877,333057,,-48739,-105924,36624,138569


In [14]:
df.to_csv("C:/Users/Administrator/Desktop/Impact_Capital/data/pf/2018_objects_part1.csv")

In [18]:
files = getListOfFiles(dirName)
files = files[235260:455834]
df = pd.concat([one_row for name in files if (one_row := analyze_one(name)) is not None])
df.to_csv("C:/Users/Administrator/Desktop/Impact_Capital/data/pf/2018_objects_part2.csv")

 52%|█████▏    | 235185/455834 [1:24:44<1:19:30, 46.25it/s]
